### 1. Import Modules and Data

In [13]:
# import modules
import numpy as np
import pandas as pd
import datetime as dt
import warnings
import os
import glob

from pypfopt.efficient_frontier import EfficientFrontier

import matplotlib
matplotlib.use("pgf")

import matplotlib.pyplot as plt

plt.rcParams.update({
    "font.family": "charter",
    "text.usetex": True,
    "pgf.rcfonts": False})

# filter warnings
warnings.filterwarnings("ignore")

In [2]:
# read in all csv-files
path = os.getcwd() + "/Data"
files = glob.glob(os.path.join(path, "*.csv"))
file_list = list()
for f in files:
    df = pd.read_csv(f)
    file_list.append(df)

In [3]:
# compute simple returns and set date as index
for i in range(len(file_list)):
    file_list[i]["Date"] = pd.to_datetime(file_list[i]["Date"])
    file_list[i].set_index("Date", inplace = True)   
    file_list[i]["Returns"] = file_list[i]["Adj Close"].pct_change()*100
    file_list[i] = file_list[i]["Returns"]
    file_list[i] = file_list[i].iloc[1:]
    file_list[i].name = files[i].split("/")[-1].split(".")[0]

In [4]:
# create dataframe
df = pd.concat([file_list[1], file_list[2], 
                file_list[3], file_list[4], 
                file_list[6]], axis = 1).dropna()

### 2. Optimization

In [5]:
# sample variance-covariance matrix
sigma = df.cov()

In [6]:
# sample vector of mean returns
mu = df.mean()

In [7]:
# mean-variance portfolio
min_vola = EfficientFrontier(mu, sigma, weight_bounds = (0,1))
min_vola.min_volatility()
weights_mv = min_vola.clean_weights()
weights_mv

OrderedDict([('ETH-USD', 0.09013),
             ('BTC-USD', 0.87538),
             ('ADA-USD', 0.0),
             ('LINK-USD', 0.02711),
             ('BNB-USD', 0.00738)])

In [8]:
# max-quadratic-utility
max_qu = EfficientFrontier(mu, sigma, weight_bounds = (0,1))
max_qu.max_quadratic_utility(risk_aversion = 0.1)
weights_qu = max_qu.clean_weights()
weights_qu

OrderedDict([('ETH-USD', 0.01956),
             ('BTC-USD', 0.74917),
             ('ADA-USD', 0.02923),
             ('LINK-USD', 0.08476),
             ('BNB-USD', 0.11729)])

In [9]:
# max sharpe-ratio
max_ms = EfficientFrontier(mu, sigma, weight_bounds = (0,1))
max_ms.max_sharpe(risk_free_rate = 0)
weights_ms = max_ms.clean_weights()
weights_ms

OrderedDict([('ETH-USD', 0.0),
             ('BTC-USD', 0.0),
             ('ADA-USD', 0.21408),
             ('LINK-USD', 0.2549),
             ('BNB-USD', 0.53102)])

### 3. Efficient Frontier

In [10]:
# generate n random portfolios for plotting
n = 10000
w = np.random.dirichlet(np.ones(len(mu)), n)

# compute returns and volatility for each possible portfolio
returns = w.dot(mu)
variance = np.sqrt(np.diag(w @ sigma @ w.T))

# compute sharpe-ratio
sharpe_ratio = (returns - 0) / variance

In [11]:
returns_mv, variance_mv, _ = min_vola.portfolio_performance()
returns_qu, variance_qu, _ = max_qu.portfolio_performance()
returns_ms, variance_ms, _ = max_ms.portfolio_performance()

In [12]:
# plot and export efficient frontier
fig = plt.figure(figsize = (12, 8))
plt.grid(axis = "y", linewidth = 0.5, linestyle = ":")
plt.scatter(variance, returns, marker = "+", c = sharpe_ratio, cmap = "plasma")
plt.scatter(variance_ms, returns_ms, marker = ".", s = 300, c = "b", label = "Maximum Sharpe-Ratio")
plt.scatter(variance_mv, returns_mv, marker = ".", s = 300, c = "g", label = "Minimum Volatility")
plt.scatter(variance_qu, returns_qu, marker = ".", s = 300, c = "r", label = "Maximum Quadratic-Utility")
plt.xlabel("Volatility")
plt.ylabel("Return")
plt.tick_params(direction = "in")
plt.legend(frameon = False, borderpad = 1.5, labelspacing = 1, prop = {"size": 9})
plt.xticks(np.arange(4.5, 8, step = 0.5))
fig.set_size_inches(w = 5.5, h = 3)
plt.savefig("ef.pgf")